In [ ]:
import gpytorch
from LODEGP.LODEGP import LODEGP, list_standard_models
from helpers.plotting_functions import plot_single_input_gp_posterior
from helpers.training_functions import granso_optimization
import matplotlib.pyplot as plt
import numpy as np
import torch



In [ ]:
list_standard_models()

In [ ]:
START = 2
END = 12
COUNT = 100
train_x = torch.linspace(START, END, COUNT)
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)

y0_func = lambda x: float(781/8000)*torch.sin(x)/x - float(1/20)*torch.cos(x)/x**2 + float(1/20)*torch.sin(x)/x**3
y1_func = lambda x: float(881/8000)*torch.sin(x)/x - float(1/40)*torch.cos(x)/x**2 + float(1/40)*torch.sin(x)/x**3
y2_func = lambda x: float(688061/800000)*torch.sin(x)/x - float(2543/4000)*torch.cos(x)/x**2 + float(1743/4000)*torch.sin(x)/x**3 - float(3/5)*torch.cos(x)/x**4 + float(3/5)*torch.sin(x)/x**5 
y0 = y0_func(train_x)
y1 = y1_func(train_x)
y2 = y2_func(train_x)
train_y = torch.stack([y0, y1, y2], dim=1)


model = LODEGP(train_x, train_y, likelihood, 3, ODE_name="Bipendulum", verbose=True, system_parameters={"l1": 1.0, "l2": 2.0})

In [ ]:
loss_val, model, likelihood, log = granso_optimization(model, likelihood, train_x, train_y)

In [ ]:
model.eval()
likelihood.eval()
with torch.no_grad():
    test_x = torch.linspace(3, 8, 10)
    output = model(test_x)
    pred_y = likelihood(output)
print(output.mean)
print(output.covariance_matrix.diag().reshape(-1, 3))
print(pred_y.mean)
print(pred_y.covariance_matrix.diag().reshape(-1, 3))

print("====")
print(list(model.named_parameters()))

In [ ]:
model_state_dict = model.state_dict()
model_state_dict

In [ ]:
# Plot the posterior GP and the data
model.eval()
model.likelihood.eval()
with torch.no_grad():
    test_x = torch.linspace(4, 12, 100)
    observed_pred = model.likelihood(model(test_x))
    observed_pred_mean = observed_pred.mean
    observed_pred_var = observed_pred.covariance_matrix.diag().reshape(-1, 3)
    print(observed_pred_mean)
    print(observed_pred_var)
    plot_single_input_gp_posterior(train_x, train_y, test_x, observed_pred_mean, observed_pred_var, ncols=2)